In [1]:
from models import *
from utils import *
import torch 
from torch import nn

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Load in the VGG16 pretrained teacher

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
t = vgg16_bn_f(False,100).to(device)
t.load_state_dict(torch.load('teacher_model_vgg16_bn.pth'))

C:\Users\PCF\AppData\Local\Temp\ipykernel_31672\1506645479.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t.load_state_dict(torch.load('teacher_model_vgg16_bn.pth'))


<All keys matched successfully>

Evaluate initial performance of teacher on test data with color jitter and test data without color jitter

In [3]:
print("Teacher Test Accuracy: ", eval_f(t, test_loader,nn.CrossEntropyLoss(),device)[1])

Teacher Test Accuracy:  0.62009996


In [4]:
print("Teacher Test Accuracy with Jitter: ", eval_f(t, color_jitter_test_loader,nn.CrossEntropyLoss(),device)[1])

Teacher Test Accuracy with Jitter:  0.51089996


Tune the teacher model on 5 epochs with color jitter train dataset (or until we regain near initial performance)

In [7]:
t.train()
EPOCHS = 5
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(t.parameters(), lr=0.0005)
for epoch in range(EPOCHS):
    independent_train_f(t, color_jitter_train_loader, criterion, optimizer, device)
    print("Epoch: ", epoch+1)
    print("Teacher Test Accuracy: ", eval_f(t, test_loader,nn.CrossEntropyLoss(),device)[1])
    print("Teacher Test Accuracy with Jitter: ", eval_f(t, color_jitter_test_loader,nn.CrossEntropyLoss(),device)[1])

Epoch:  1
Teacher Test Accuracy:  0.6501
Teacher Test Accuracy with Jitter:  0.6062
Epoch:  2
Teacher Test Accuracy:  0.6518
Teacher Test Accuracy with Jitter:  0.61539996
Epoch:  3
Teacher Test Accuracy:  0.6563
Teacher Test Accuracy with Jitter:  0.6192
Epoch:  4
Teacher Test Accuracy:  0.66179997
Teacher Test Accuracy with Jitter:  0.6232
Epoch:  5
Teacher Test Accuracy:  0.6551
Teacher Test Accuracy with Jitter:  0.62229997


Save this color invariant teacher model

In [8]:
torch.save(t.state_dict(), 'teacher_model_vgg16_bn_color_inv.pth')

Train a CRD with regular augmentations on the invariant model

In [10]:
teacher = vgg16_bn_f(False,100).to(device)
teacher.load_state_dict(torch.load('teacher_model_vgg16_bn_color_inv.pth')) 
student = vgg11_bn_f(False,100).to(device)  
crd_net = create_crd_model(teacher, student)
crd_net.student.to(device)
crd_net.teacher.to(device)
crd_net.crd_block.to(device)
EPOCHS = 30
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(get_learnable_parameters(crd_net), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}


for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_crd(crd_net,train_loader_idx,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(crd_net.student,test_loader,nn.CrossEntropyLoss(),device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
ts_color_loss, ts_color_acc = eval_f(crd_net.student,color_jitter_test_loader,nn.CrossEntropyLoss(),device)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))
print("Epoch: {} Test Color Jitter Loss: {:.2f} Test Color Jitter Accuracy: {:.2f}".format(epoch+1, ts_color_loss, ts_color_acc))

C:\Users\PCF\AppData\Local\Temp\ipykernel_31672\1999419255.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load('teacher_model_vgg16_bn_col

Epoch: 1 Train Loss: 12.92 Train Accuracy: 0.03
Epoch: 2 Train Loss: 12.82 Train Accuracy: 0.05
Epoch: 3 Train Loss: 12.75 Train Accuracy: 0.06
Epoch: 4 Train Loss: 12.68 Train Accuracy: 0.07
Epoch: 5 Train Loss: 12.59 Train Accuracy: 0.10
Epoch: 6 Train Loss: 12.51 Train Accuracy: 0.12
Epoch: 7 Train Loss: 12.44 Train Accuracy: 0.14
Epoch: 8 Train Loss: 12.38 Train Accuracy: 0.16
Epoch: 9 Train Loss: 12.33 Train Accuracy: 0.18
Epoch: 10 Train Loss: 12.28 Train Accuracy: 0.20
Epoch: 11 Train Loss: 12.23 Train Accuracy: 0.22
Epoch: 12 Train Loss: 12.19 Train Accuracy: 0.24
Epoch: 13 Train Loss: 12.15 Train Accuracy: 0.25
Epoch: 14 Train Loss: 12.11 Train Accuracy: 0.27
Epoch: 15 Train Loss: 12.08 Train Accuracy: 0.29
Epoch: 16 Train Loss: 12.05 Train Accuracy: 0.30
Epoch: 17 Train Loss: 12.02 Train Accuracy: 0.32
Epoch: 18 Train Loss: 11.99 Train Accuracy: 0.33
Epoch: 19 Train Loss: 11.96 Train Accuracy: 0.35
Epoch: 20 Train Loss: 11.93 Train Accuracy: 0.36
Epoch: 21 Train Loss: 11.91 T

In [11]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])
print("Final Test Color Jitter Accuracy:",ts_color_acc)
print("Final Test Color Jitter Loss:",ts_color_loss)

Final Train Accuracy: 0.4881
Final Test Accuracy: 0.4181
Final Train Loss: 11.707672150848467
Final Test Loss: 2.3314421161724503
Final Test Color Jitter Accuracy: 0.3216
Final Test Color Jitter Loss: 3.0349377629103933


In [12]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_crd_color_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_crd_color_ts.csv")

In [13]:
torch.save(crd_net.student.state_dict(),"vgg11_cifar100_crd_color_student.pth")
torch.save(crd_net.crd_block.state_dict(),"vgg11_cifar100_crd_color_embedder.pth")

Compare this with the color jitter accuracy of regular augmentations trained CRD student

In [14]:
student_crd = vgg11_bn_f(False,100).to(device)
student_crd.load_state_dict(torch.load('vgg11_cifar100_crd_student.pth'))
student_crd.eval()
print("Student Test Accuracy: ", eval_f(student_crd, test_loader,nn.CrossEntropyLoss(),device)[1])
print("Student Test Accuracy with Jitter: ", eval_f(student_crd, color_jitter_test_loader,nn.CrossEntropyLoss(),device)[1])

C:\Users\PCF\AppData\Local\Temp\ipykernel_31672\1082311228.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student_crd.load_state_dict(torch.load('vgg11_cifar100_crd_stu

Student Test Accuracy:  0.47059998
Student Test Accuracy with Jitter:  0.3362


Repeat the procedure with FitNets (Hint)

In [15]:
VGG16_teacher = vgg16_bn_f(False,100)
VGG16_teacher.load_state_dict(torch.load('teacher_model_vgg16_bn_color_inv.pth'))

VGG11_student = vgg11_bn_f(False,100)

fitnet = create_model(VGG16_teacher, VGG11_student, 3,0.75)

fitnet.student.to(device)
fitnet.teacher.to(device)
fitnet.regressor = fitnet.regressor.to(device)

EPOCHS = 30
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fitnet.get_learnable_params(), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_hints(fitnet,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_hints(fitnet,test_loader,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


C:\Users\PCF\AppData\Local\Temp\ipykernel_31672\1016480240.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  VGG16_teacher.load_state_dict(torch.load('teacher_model_vgg16_

Epoch: 1 Train Loss: 1.09 Train Accuracy: 0.03
Epoch: 2 Train Loss: 1.04 Train Accuracy: 0.04
Epoch: 3 Train Loss: 1.01 Train Accuracy: 0.05
Epoch: 4 Train Loss: 0.99 Train Accuracy: 0.06
Epoch: 5 Train Loss: 0.96 Train Accuracy: 0.08
Epoch: 6 Train Loss: 0.93 Train Accuracy: 0.09
Epoch: 7 Train Loss: 0.91 Train Accuracy: 0.10
Epoch: 8 Train Loss: 0.88 Train Accuracy: 0.12
Epoch: 9 Train Loss: 0.86 Train Accuracy: 0.13
Epoch: 10 Train Loss: 0.84 Train Accuracy: 0.14
Epoch: 11 Train Loss: 0.82 Train Accuracy: 0.16
Epoch: 12 Train Loss: 0.81 Train Accuracy: 0.17
Epoch: 13 Train Loss: 0.79 Train Accuracy: 0.18
Epoch: 14 Train Loss: 0.77 Train Accuracy: 0.20
Epoch: 15 Train Loss: 0.75 Train Accuracy: 0.21
Epoch: 16 Train Loss: 0.74 Train Accuracy: 0.22
Epoch: 17 Train Loss: 0.72 Train Accuracy: 0.24
Epoch: 18 Train Loss: 0.71 Train Accuracy: 0.25
Epoch: 19 Train Loss: 0.69 Train Accuracy: 0.26
Epoch: 20 Train Loss: 0.68 Train Accuracy: 0.27
Epoch: 21 Train Loss: 0.67 Train Accuracy: 0.28
E

In [16]:
ts_color_loss, ts_color_acc = eval_f(fitnet.student,color_jitter_test_loader,nn.CrossEntropyLoss(),device)
print("Epoch: {} Test Color Jitter Loss: {:.2f} Test Color Jitter Accuracy: {:.2f}".format(epoch+1, ts_color_loss, ts_color_acc))
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])
print("Final Test Color Jitter Accuracy:",ts_color_acc)
print("Final Test Color Jitter Loss:",ts_color_loss)

Epoch: 30 Test Color Jitter Loss: 3.03 Test Color Jitter Accuracy: 0.26
Final Train Accuracy: 0.39292
Final Test Accuracy: 0.3917
Final Train Loss: 0.5563796124494899
Final Test Loss: 2.2606180450718876
Final Test Color Jitter Accuracy: 0.2581
Final Test Color Jitter Loss: 3.0270283692961284


In [17]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.39292
Final Test Accuracy: 0.3917
Final Train Loss: 0.5563796124494899
Final Test Loss: 2.2606180450718876


In [18]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_hint_color_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_hint_color_ts.csv")
torch.save(fitnet.student.state_dict(),"vgg11_cifar100_hint_student_color.pth")
torch.save(fitnet.regressor.state_dict(),"vgg11_cifar100_hint_regressor_color.pth")

Compare this with the color jitter accuracy of regular augmentations trained CRD student

In [19]:
student_hint = vgg11_bn_f(False,100).to(device)
student_hint.load_state_dict(torch.load('vgg11_cifar100_hint_student.pth'))
student_hint.eval()
print("Student Test Accuracy: ", eval_f(student_hint, test_loader,nn.CrossEntropyLoss(),device)[1])
print("Student Test Accuracy with Jitter: ", eval_f(student_hint, color_jitter_test_loader,nn.CrossEntropyLoss(),device)[1])

C:\Users\PCF\AppData\Local\Temp\ipykernel_31672\1129106845.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student_hint.load_state_dict(torch.load('vgg11_cifar100_hint_s

Student Test Accuracy:  0.4353
Student Test Accuracy with Jitter:  0.3011


Repeat the procedure with Basic Logit Smoothing (Hinton)

In [20]:
student_model = vgg11_bn_f(False,100)
teacher_model_vgg16_bn = vgg16_bn_f(False,100)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_model_vgg16_bn_color_inv.pth")) 
for param in student_model.parameters():
    param.requires_grad = True          
for param in teacher_model_vgg16_bn.parameters():
    param.requires_grad = False
teacher_model = teacher_model_vgg16_bn.to(device)
student_model = student_model.to(device)
optimizer = torch.optim.Adam(student_model.parameters(),lr=0.002)
loss_func = torch.nn.CrossEntropyLoss()
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in (range(EPOCHS)):
    tr_loss, tr_acc = train(student_model,teacher_model,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_f(student_model,test_loader,loss_func,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))
ts_color_loss, ts_color_acc = eval_f(fitnet.student,color_jitter_test_loader,nn.CrossEntropyLoss(),device)
print("Epoch: {} Test Color Jitter Loss: {:.2f} Test Color Jitter Accuracy: {:.2f}".format(epoch+1, ts_color_loss, ts_color_acc))

C:\Users\PCF\AppData\Local\Temp\ipykernel_31672\948026561.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model_vgg16_bn.load_state_dict(torch.load("teacher_mode

Epoch: 1 Train Loss: 8.20 Train Accuracy: 0.03
Epoch: 2 Train Loss: 7.70 Train Accuracy: 0.05
Epoch: 3 Train Loss: 7.32 Train Accuracy: 0.07
Epoch: 4 Train Loss: 6.93 Train Accuracy: 0.09
Epoch: 5 Train Loss: 6.59 Train Accuracy: 0.11
Epoch: 6 Train Loss: 6.28 Train Accuracy: 0.13
Epoch: 7 Train Loss: 5.98 Train Accuracy: 0.16
Epoch: 8 Train Loss: 5.72 Train Accuracy: 0.18
Epoch: 9 Train Loss: 5.48 Train Accuracy: 0.20
Epoch: 10 Train Loss: 5.27 Train Accuracy: 0.22
Epoch: 11 Train Loss: 5.08 Train Accuracy: 0.24
Epoch: 12 Train Loss: 4.91 Train Accuracy: 0.25
Epoch: 13 Train Loss: 4.76 Train Accuracy: 0.27
Epoch: 14 Train Loss: 4.59 Train Accuracy: 0.29
Epoch: 15 Train Loss: 4.41 Train Accuracy: 0.31
Epoch: 16 Train Loss: 4.28 Train Accuracy: 0.32
Epoch: 17 Train Loss: 4.16 Train Accuracy: 0.34
Epoch: 18 Train Loss: 4.03 Train Accuracy: 0.36
Epoch: 19 Train Loss: 3.90 Train Accuracy: 0.37
Epoch: 20 Train Loss: 3.81 Train Accuracy: 0.38
Epoch: 21 Train Loss: 3.70 Train Accuracy: 0.40
E

In [21]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])
print("Final Test Color Jitter Accuracy:",ts_color_acc)
print("Final Test Color Jitter Loss:",ts_color_loss)

Final Train Accuracy: 0.51063997
Final Test Accuracy: 0.4574
Final Train Loss: 2.9687367715798985
Final Test Loss: 2.2481329828310925
Final Test Color Jitter Accuracy: 0.2602
Final Test Color Jitter Loss: 3.054111845174413


In [22]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_blm_color_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_blm_color_ts.csv")
torch.save(student_model.state_dict(),"vgg11_cifar100_blm_color.pth")

Compare this with the color jitter accuracy of regular augmentations trained CRD student

In [23]:
student_hint = vgg11_bn_f(False,100).to(device)
student_hint.load_state_dict(torch.load('vgg11_cifar100_blm.pth'))
student_hint.eval()
print("Student Test Accuracy: ", eval_f(student_hint, test_loader,nn.CrossEntropyLoss(),device)[1])
print("Student Test Accuracy with Jitter: ", eval_f(student_hint, color_jitter_test_loader,nn.CrossEntropyLoss(),device)[1])

C:\Users\PCF\AppData\Local\Temp\ipykernel_31672\2422101631.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student_hint.load_state_dict(torch.load('vgg11_cifar100_blm.pt

Student Test Accuracy:  0.5202
Student Test Accuracy with Jitter:  0.3895
